In [1]:
import pandas as pd
import yaml

In [3]:
languages = pd.read_csv("languages.csv")
print(languages)

            repo_name  is_private         created_date  is_archived  language  \
0      animal_mosaics        True  2020-12-26 04:56:14         True      HTML   
1      animal_mosaics        True  2020-12-26 04:56:14         True       CSS   
2      animal_mosaics        True  2020-12-26 04:56:14         True  Markdown   
3        AppCenso2011        True  2020-05-23 16:33:25         True         R   
4        AppCenso2011        True  2020-05-23 16:33:25         True    Python   
..                ...         ...                  ...          ...       ...   
376  udemy_sales_dash       False  2021-02-23 23:58:18         True    Python   
377  udemy_sales_dash       False  2021-02-23 23:58:18         True       CSS   
378  udemy_sales_dash       False  2021-02-23 23:58:18         True  Markdown   
379            vierci        True  2019-09-17 21:37:20         True         R   
380            vierci        True  2019-09-17 21:37:20         True  Markdown   

      size  
0     5715  
1

In [16]:
with open(f"language_colors.yaml", "r") as yaml_file:
  colors_yaml = yaml.safe_load(yaml_file)
colors = pd.DataFrame.from_dict({
  "language": colors_yaml.keys(),
  "color": colors_yaml.values()
})
print(colors)

          language    color
0    1C Enterprise  #814ccc
1               4D  #004289
2             ABAP  #e8274b
3         ABAP CDS  #555e25
4             ABNF  #cccccc
..             ...      ...
607     robots.txt  #cccccc
608            sed  #64b970
609            wdl  #42f1f4
610           wisp  #7582d1
611          xBase  #403a40

[612 rows x 2 columns]


In [17]:
languages.merge(
  colors,
  how = "left",
  on = "language"
)

AttributeError: 'dict_keys' object has no attribute 'merge'